In [1]:
%matplotlib inline

import geopandas as gpd
import numpy as np
from pysal.viz.splot.libpysal import plot_spatial_weights
from libpysal.weights import Queen, Rook
from region.max_p_regions.heuristics import MaxPRegionsHeu
from region.p_regions.azp import *
from region.csgraph_utils import *
import scipy
import scipy.sparse.csgraph as csg

gdf = gpd.read_file('reg_lat_10_10.shp')

np.random.seed(123)
gdf['pop'] = np.random.randint(500, high=1000, size=100)
np.random.seed(123)
gdf['income_pp'] = np.random.uniform(low=50000, high=100000, size=100)

w_queen = Queen.from_dataframe(gdf)
w_rook = Rook.from_dataframe(gdf)
adj = scipy.sparse.csr_matrix(w_rook.full()[0])

values_gdf = gdf[['income_pp']]
spatially_extensive_attr_gdf = gdf[['pop']].values

In [2]:
from joblib import Parallel, delayed
import multiprocessing

Draw inspiration from https://blog.dominodatalab.com/simple-parallelization/

In [3]:
inputs = range(100, 125)
threshold = 10000

In [4]:
num_cores = multiprocessing.cpu_count()

In [5]:
def processInput(i):
    model = MaxPRegionsHeu(local_search = AZP(), random_state=i)
    model.fit_from_w(w_rook, values_gdf.values, spatially_extensive_attr_gdf, threshold = threshold)
    gdf['labels'] = model.labels_
    return [any(gdf.groupby('labels')['pop'].sum() < threshold), not boolean_assert_feasible(gdf['labels'], adj)]
     
results = Parallel(n_jobs=num_cores)(delayed(processInput)(i) for i in inputs)

thr_aux = [l[0] for l in results]
con_aux = [l[1] for l in results]

print('Problems in Threshold: {}'.format(any(thr_aux)))
print('Problems in Contiguity: {}'.format(any(con_aux)))

Problems in Threshold: False
Problems in Contiguity: False


In [6]:
AZP_thr_aux = thr_aux
AZP_con_aux = con_aux

In [7]:
def processInput(i):
    model = MaxPRegionsHeu(local_search = AZPBasicTabu(), random_state=i)
    model.fit_from_w(w_rook, values_gdf.values, spatially_extensive_attr_gdf, threshold = threshold)
    gdf['labels'] = model.labels_
    return [any(gdf.groupby('labels')['pop'].sum() < threshold), not boolean_assert_feasible(gdf['labels'], adj)]
     
results = Parallel(n_jobs=num_cores)(delayed(processInput)(i) for i in inputs)

thr_aux = [l[0] for l in results]
con_aux = [l[1] for l in results]

print('Problems in Threshold: {}'.format(any(thr_aux)))
print('Problems in Contiguity: {}'.format(any(con_aux)))

Problems in Threshold: False
Problems in Contiguity: False


In [8]:
AZPBasicTabu_thr_aux = thr_aux
AZPBasicTabu_con_aux = con_aux

In [9]:
def processInput(i):
    model = MaxPRegionsHeu(local_search = AZPReactiveTabu(max_iterations = 2, k1 = 2, k2 = 2), random_state=i)
    model.fit_from_w(w_rook, values_gdf.values, spatially_extensive_attr_gdf, threshold = threshold)
    gdf['labels'] = model.labels_
    return [any(gdf.groupby('labels')['pop'].sum() < threshold), not boolean_assert_feasible(gdf['labels'], adj)]
     
results = Parallel(n_jobs=num_cores)(delayed(processInput)(i) for i in inputs)

thr_aux = [l[0] for l in results]
con_aux = [l[1] for l in results]

print('Problems in Threshold: {}'.format(any(thr_aux)))
print('Problems in Contiguity: {}'.format(any(con_aux)))

Problems in Threshold: False
Problems in Contiguity: False


In [10]:
AZPReactiveTabu_thr_aux = thr_aux
AZPReactiveTabu_con_aux = con_aux

In [11]:
def processInput(i):
    model = MaxPRegionsHeu(local_search = AZPSimulatedAnnealing(init_temperature=1), random_state=i)
    model.fit_from_w(w_rook, values_gdf.values, spatially_extensive_attr_gdf, threshold = threshold)
    gdf['labels'] = model.labels_
    return [any(gdf.groupby('labels')['pop'].sum() < threshold), not boolean_assert_feasible(gdf['labels'], adj)]
     
results = Parallel(n_jobs=num_cores)(delayed(processInput)(i) for i in inputs)

thr_aux = [l[0] for l in results]
con_aux = [l[1] for l in results]

print('Problems in Threshold: {}'.format(any(thr_aux)))
print('Problems in Contiguity: {}'.format(any(con_aux)))

Problems in Threshold: False
Problems in Contiguity: False


In [12]:
AZPSimulatedAnnealing_thr_aux = thr_aux
AZPSimulatedAnnealing_con_aux = con_aux